In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, concat, read_csv
sns.set_theme(style='white', context='notebook', font_scale=1.33)

## Section 1: Variance Decomposition

In [2]:
studies = ['teicher2015', 'tuominen2022', 'joint']

## Load design data.
design = read_csv(os.path.join('data', 'design.csv'), index_col=0)
design = design[design.columns[:11]]

stats = []
for study in studies:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load Stan summary.
    summary = read_csv(os.path.join('stan_results', study, '2plq_m2_summary.tsv'), sep='\t', index_col=0)
    
    ## Extract factor loadings.
    loadings = np.zeros_like(design).astype(float)
    for i, j in np.column_stack([np.where(design)]).T:
        loadings[i,j] = summary.loc[f'lambda[{i+1},{j+1}]','Mean']
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Coefficient omega hierachical.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Preallocate space.
    omega   = np.zeros(len(design.columns))
    omega_s = np.zeros(len(design.columns))
        
    ## Iterate over factors.
    for i, col in enumerate(design.columns):
        
        ## Restrict to items in group.
        L = loadings[design[col]==1]
        
        ## Compute squared sum of factor loadings.
        A = np.square(np.sum(L, axis=0))
        
        ## Compute sum of error variances.
        B = np.sum(1 - np.square(L).sum(axis=1))
        
        ## Compute total variance.
        C = np.sum(A) + B
        
        ## Compute coefficient omega.
        omega[i] = A.sum() / C
        
        ## Compute coefficient omega subscale.
        omega_s[i] = A[i] / C
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Explained common variance.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Compute sum of squares.
    ss = np.square(loadings).sum(axis=0)
    
    ## Compute explained common variance.
    ecv = ss / ss.sum()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### H-index
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Preallocate space.
    H = np.zeros(len(design.columns))
    
    ## Iterate over factors.
    for i, col in enumerate(design.columns):
        
        ## Compute squared loadings.
        s = np.square(loadings[:,i])
        
        ## Compute H-index.
        H[i] = 1. / (1 + 1 / np.sum(s / (1-s)))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Convert to DataFrame.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats.append(DataFrame(dict(
        subscale = design.columns,
        study = np.repeat(study, design.columns.size),
        ecv = ecv,
        omega = omega,
        omega_s = omega_s,
        H = H
    )))
    
## Concatenate DataFrames.
stats = concat(stats).replace({'teicher2015':1, 'tuominen2022': 2, 'joint': 3})

## Convert to pivot table.
stats = stats.pivot_table(['omega','omega_s','ecv','H'], 'subscale', 'study')

## Re-organize table.
index = ['general', 'PVA', 'PPhysA', 'NVEA', 'SexA', 'EN', 'PN', 'WSV', 'WIPV', 'PeerVA', 'PeerPhysA']
stats = stats.loc[index, ['omega','omega_s','ecv','H']]

In [3]:
stats.round(3)

omega               omega_s                  ecv                \
study          1      2      3       1      2      3      1      2      3   
subscale                                                                    
general    0.982  0.981  0.981   0.925  0.916  0.915  0.584  0.576  0.568   
PVA        0.917  0.904  0.905   0.089  0.159  0.156  0.011  0.019  0.019   
PPhysA     0.924  0.917  0.916   0.302  0.431  0.422  0.048  0.062  0.061   
NVEA       0.854  0.827  0.828   0.146  0.120  0.124  0.025  0.019  0.019   
SexA       0.919  0.919  0.920   0.280  0.520  0.505  0.057  0.080  0.078   
EN         0.817  0.908  0.907   0.356  0.270  0.274  0.041  0.039  0.040   
PN         0.900  0.878  0.877   0.365  0.206  0.212  0.045  0.026  0.026   
WSV        0.849  0.816  0.816   0.088  0.111  0.217  0.008  0.009  0.019   
WIPV       0.939  0.915  0.914   0.427  0.378  0.375  0.052  0.046  0.045   
PeerVA     0.888  0.869  0.869   0.594  0.548  0.546  0.062  0.058  0.057   
PeerPhysA  0.906  0.899  0.899   0.615  0.583  0.593  0.068  0.066  0.067   

               H                
study          1      2      3  
subscale                        
general    0.977  0.972  0.971  
PVA        0.311  0.463  0.462  
PPhysA     0.789  0.828  0.823  
NVEA       0.568  0.443  0.450  
SexA       0.856  0.854  0.856  
EN         0.759  0.725  0.729  
PN         0.754  0.542  0.544  
WSV        0.219  0.235  0.453  
WIPV       0.743  0.710  0.701  
PeerVA     0.818  0.784  0.783  
PeerPhysA  0.853  0.828  0.832

In [4]:
print(stats.round(3).to_latex())

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} & \multicolumn{3}{l}{omega} & \multicolumn{3}{l}{omega\_s} & \multicolumn{3}{l}{ecv} & \multicolumn{3}{l}{H} \\
study &      1 &      2 &      3 &       1 &      2 &      3 &      1 &      2 &      3 &      1 &      2 &      3 \\
subscale  &        &        &        &         &        &        &        &        &        &        &        &        \\
\midrule
general   &  0.982 &  0.981 &  0.981 &   0.925 &  0.916 &  0.915 &  0.584 &  0.576 &  0.568 &  0.977 &  0.972 &  0.971 \\
PVA       &  0.917 &  0.904 &  0.905 &   0.089 &  0.159 &  0.156 &  0.011 &  0.019 &  0.019 &  0.311 &  0.463 &  0.462 \\
PPhysA    &  0.924 &  0.917 &  0.916 &   0.302 &  0.431 &  0.422 &  0.048 &  0.062 &  0.061 &  0.789 &  0.828 &  0.823 \\
NVEA      &  0.854 &  0.827 &  0.828 &   0.146 &  0.120 &  0.124 &  0.025 &  0.019 &  0.019 &  0.568 &  0.443 &  0.450 \\
SexA      &  0.919 &  0.919 &  0.920 &   0.280 &  0.520 &  0.505 &  0.057 &  0.080 &  0.078 &  0.856 &  0.85